# Clone

In [0]:
import os

def clone(url, dpath=".", branch="master"):
  """
  Clone remote branch from url into dpath
  branch not provided: clone all branches
  """

  url = url.strip("/")
  rname = os.path.basename(url)
  rpath = os.path.join(dpath, rname)

  # Raise error if dpath inside existing repository
  is_out = !git -C "$dpath" rev-parse
  if not is_out: # inside repository
    raise ValueError("Can't clone into existing repository")
  
  # Clone specific branch
  if branch:
    !git clone --single-branch --branch "$branch" "$url" "$rpath"
  # Clone all branches
  else:
    !git clone "$url" "$rpath"
  os.chdir(rpath)
  
  bname = !git rev-parse --abbrev-ref HEAD
  print("Current")
  print("{branch}\t{directory}".format(branch=bname, directory=os.getcwd()))
  
# clone(URL)

# Push

In [0]:
from google.colab import files
import json
from urllib.parse import urlsplit

def push(url, branch="master"):
  """
  Push branch to url
  User provides .json file containing account information
  .json file: {"username": <username>, "email": <email>, "password": <password>, "name": <display name>}
  branch not provided: push all branches
  """
  
  # Obtain account information
  uploaded = files.upload() # upload .json file
  [fpath] = [*uploaded] # key of dict uploaded is filepath
  with open(fpath) as f:
    account = json.load(f)
  !rm "$fpath" # remove .json file
  account.setdefault("name", account["username"]) # name defaults to username if not provided
  
  # Construct push-authorized url by adding username and password to repository url
  r = urlsplit(url)
  r_auth = r._replace(netloc="{username}:{password}@{hostname}".format(username=account["username"], password=account["password"], hostname=r.hostname))
  url_auth = r_auth.geturl()

  # Set up git
  name, email = account["name"], account["email"]
  !git config --global user.name "$name"
  !git config --global user.email "$email"
  !git remote add origin_auth "$url_auth"
  
  # Push
  if branch:
    input("Press Enter to push {}...".format(branch))
    !git push origin_auth "$branch"
  else:
    input("Press Enter to push all branches...")
    !git push --all origin_auth